In [1]:
import numpy as np
import torch

train = np.array([[0,0],[0,1],[1,0],[1,1]])
label = np.array([[0],[1],[1],[0]])

train_x = torch.Tensor(train)
train_y = torch.Tensor(label)

In [2]:
import torch.nn as nn

class MLP(nn.Module):
    """
    l1: hidden layer
    l2: output
    a1: activation of l1
    a2: activation of l2
    """
    def __init__(self, input_dim, hidden_dim, output__dim):
        # run init of superclass
        super().__init__()
        self.fc1 = nn.Linear(
            input_dim,
            hidden_dim,
        )
        self.fc2 = nn.Linear(
            hidden_dim,
            output__dim,
        )
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(2,2,1).to(device)

model

    

MLP(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)

In [3]:
import torch.optim
crierion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [4]:
def train_step(x, t):
    model.train()
    outputs = model(x)
    loss = crierion(outputs, t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss

In [6]:
epochs = 4000
for epoch in range(epochs):
    epoch_loss = 0.
    train_x, train_y = train_x.to(device), train_y.to(device) 

    loss = train_step(train_x, train_y)
    epoch_loss += loss.item()

    if(epoch+1)%1000 == 0:
        print("epoch({}) loss:{:.4f}".format(epoch+1, epoch_loss))

epoch(1000) loss:0.4882
epoch(2000) loss:0.0256
epoch(3000) loss:0.0112
epoch(4000) loss:0.0071


In [7]:
outputs = model(train_x)
print(outputs)
preds = (outputs.to("cpu").detach().numpy().copy() > 0.5).astype(np.int32)

print(preds)

tensor([[0.0086],
        [0.9936],
        [0.9936],
        [0.0068]], grad_fn=<SigmoidBackward0>)
[[0]
 [1]
 [1]
 [0]]
